## 이미지 파일 확인

In [27]:
import os
file_path = "/Users/nayoungmin/Desktop/Capstone/image_yeju"
file_names = [f for f in os.listdir(file_path) if os.path.isdir(os.path.join(file_path, f))]

file_Path = []
# 장소별 데이터 수
for place in file_names:
    PATH = os.path.join(file_path, place)
    print(str(place), ":", str(len(os.listdir(PATH))))
    file_Path.append(PATH)

print(file_Path)


벚꽃 : 442
놀이공원 : 348
타워 : 318
유람선 : 366
케이블카 : 307
빌딩 : 454
노을 : 393
번화가 : 335
돌담길 : 704
궁 : 363
철로 : 434
자연 : 705
아쿠아리움 : 334
피크닉 : 410
아이스링크 : 82
단풍 : 341
전통문화 : 301
야경 : 431
도심하천 : 791
극장 : 865
워터파크 : 320
['/Users/nayoungmin/Desktop/Capstone/image_yeju/벚꽃', '/Users/nayoungmin/Desktop/Capstone/image_yeju/놀이공원', '/Users/nayoungmin/Desktop/Capstone/image_yeju/타워', '/Users/nayoungmin/Desktop/Capstone/image_yeju/유람선', '/Users/nayoungmin/Desktop/Capstone/image_yeju/케이블카', '/Users/nayoungmin/Desktop/Capstone/image_yeju/빌딩', '/Users/nayoungmin/Desktop/Capstone/image_yeju/노을', '/Users/nayoungmin/Desktop/Capstone/image_yeju/번화가', '/Users/nayoungmin/Desktop/Capstone/image_yeju/돌담길', '/Users/nayoungmin/Desktop/Capstone/image_yeju/궁', '/Users/nayoungmin/Desktop/Capstone/image_yeju/철로', '/Users/nayoungmin/Desktop/Capstone/image_yeju/자연', '/Users/nayoungmin/Des

## GPU 사용

## 모듈 불러오기

### GPU 확인

In [28]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        print("Cannot set memory growth on device")
    else:
        print("GPU memory growth set to True")
else:
    print("Not enough GPU hardware devices available")


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## train, validation 나누기
7.5:2.5 비율로 나눔

In [1]:
import splitfolders   # 입력된 비율을 기준으로 데이터셋을 분할하는 함수
input_folder = "/Users/nayoungmin/Desktop/Capstone/image_yeju"
output = "/Users/nayoungmin/Desktop/Capstone/splited_image_is" # where you want the split datasets saved. one will be created if it does not exist or none is set

splitfolders.ratio(input_folder, output=output, seed=777, ratio=(0.75, 0.25)) # 7.5:2.5

## class별 데이터 수 확인

In [2]:
import os

file_path = '/Users/nayoungmin/Desktop/Capstone/splited_image_is'
file_names = [f for f in os.listdir(file_path) if not f.startswith('.')]  # 숨김파일 제외

for data in file_names:
    PATH = os.path.join(file_path,data)
    plan_file = [f for f in os.listdir(PATH) if not f.startswith('.')]  # 숨김파일 제외
    print("[",str(data),"data ]")
    print()
    for plan in plan_file:
        PATH_ = os.path.join(PATH,plan)
        print(str(plan)," : ",str(len(os.listdir(PATH_))))
    print()

[ train data ]

벚꽃  :  330
놀이공원  :  261
타워  :  237
유람선  :  274
케이블카  :  230
빌딩  :  339
노을  :  294
번화가  :  251
트리  :  258
돌담길  :  527
호수공원  :  306
궁  :  272
철로  :  324
자연  :  528
아쿠아리움  :  250
아이스링크  :  61
단풍  :  255
전통문화  :  225
야경  :  323
도심하천  :  592
극장  :  648
워터파크  :  240

[ val data ]

벚꽃  :  111
놀이공원  :  87
타워  :  80
유람선  :  92
케이블카  :  77
빌딩  :  114
노을  :  99
번화가  :  84
트리  :  87
돌담길  :  176
호수공원  :  103
궁  :  91
철로  :  109
자연  :  176
아쿠아리움  :  84
아이스링크  :  21
단풍  :  86
전통문화  :  76
야경  :  108
도심하천  :  198
극장  :  216
워터파크  :  80



## 이미지 전처리 (이미지 증강 기법 사용)

In [3]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_PATH = '/Users/nayoungmin/Desktop/Capstone/splited_image_is/train'
VAL_PATH = '/Users/nayoungmin/Desktop/Capstone/splited_image_is/val'


MODEL_PATH = '/Users/nayoungmin/Desktop/Capstone/model'

BATCH_SIZE = 50
IMG_HEIGHT = 224  
IMG_WIDTH = 224 
N_EPOCH = 15
LR = 0.05
nb_classes = 82

trainGen = ImageDataGenerator(
    rescale=1./255,        # 값을 0과 1 사이로 변경
    rotation_range=30,      # 무작위 회전각도 30도 이내
    shear_range=0.2,         # 층밀리기 강도 20% (정사각형 -> 평행사변형)
    zoom_range=0.2,            # 무작위 줌 범위 20%
    horizontal_flip=True    # 무작위로 가로로 뒤짚는다.
)
valGen = ImageDataGenerator(
    rescale=1./255,       
    rotation_range=30,   
    shear_range=0.2,      
    zoom_range=0.2,            
    horizontal_flip=True    
)

train_generator = trainGen.flow_from_directory(      # flow_from_directory() : 각 이미지마다 디렉토리 이름을 레이블로 사용
   TRAIN_PATH,
   class_mode="categorical",
   target_size=(IMG_HEIGHT, IMG_WIDTH),
   shuffle=True,
   batch_size=BATCH_SIZE)

# initialize the validation generator
validation_generator = valGen.flow_from_directory(  # flow_from_directory() : 각 이미지마다 디렉토리 이름을 레이블로 사용
   VAL_PATH,
   class_mode="categorical",
   target_size=(IMG_HEIGHT, IMG_WIDTH),
   shuffle=True,
   batch_size=BATCH_SIZE)


Found 7025 images belonging to 22 classes.
Found 2355 images belonging to 22 classes.


## 모델 생성 및 학습

In [4]:
from keras.layers import AveragePooling2D
from keras.layers  import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

from keras.metrics import AUC

from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions


# ResNet50V2 모델을 불러오고, 그 모델의 출력을 활용하여 새로운 fully connected layer를 추가
resnet = ResNet50V2(include_top=False, weights='imagenet', input_shape = (224,224,3))

x = resnet.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x =  Dense(512, activation='relu', input_dim= (224,224,3))(x)
x = BatchNormalization()(x)
x =  Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x =  Dense(22, activation='softmax')(x)  # 여기에서 Dense를 class 수에 맞게 설정해야 함.

model = Model(inputs=resnet.input, outputs=x)

# 기존 학습된 layer들을 freazing(새로운 layer들만 학습하도록)
for layer in resnet.layers:
   layer.trainable = False

# define the learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch % 10 == 0:
        lr = lr * 0.1
    return lr

# compile the model with the learning rate scheduler
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy",AUC(multi_label=True,num_labels=22,name='AUC')])

# train the model
H = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=14,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)]
)

model.save('my_model_final.h5')

Metal device set to: Apple M1 Pro


/var/folders/0q/f448gzfd2rd9vk0q5h6wlvn80000gn/T/ipykernel_2779/4159987933.py:56: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/14


2023-04-11 14:38:00.761327: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


140/140 [==============================] - 63s 436ms/step - loss: 0.8442 - accuracy: 0.7508 - AUC: 0.9710 - val_loss: 0.5340 - val_accuracy: 0.8485 - val_AUC: 0.9862 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/14
140/140 [==============================] - 57s 406ms/step - loss: 0.4526 - accuracy: 0.8594 - AUC: 0.9907 - val_loss: 0.3942 - val_accuracy: 0.8715 - val_AUC: 0.9914 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/14
140/140 [==============================] - 58s 414ms/step - loss: 0.3650 - accuracy: 0.8834 - AUC: 0.9936 - val_loss: 0.4080 - val_accuracy: 0.8838 - val_AUC: 0.9891 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/14
140/140 [==============================] - 57s 406ms/step - loss: 0.3162 - accuracy: 0.9009 - AUC: 0.9949 - val_loss: 0.4014 - val_accuracy: 0.8774 - val_AUC: 0.9896 - lr: 0.0010

Epoch 5: L

## 예측
이미지 넣어서 예측해보기

In [25]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions

img_path = '/Users/nayoungmin/Desktop/스크린샷 2023-04-11 오후 3.46.20.png' # 예측하고자 하는 이미지 경로
target_size = (224, 224) # 모델이 학습시킨 이미지 사이즈와 동일하게 설정

# 이미지 전처리
img = load_img(img_path, target_size=target_size)
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0 # 이미지 스케일링

# 예측
preds = model.predict(x)

# 클래스별 확률 값 출력
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

for i in range(len(class_names)):
  print(f"{class_names[i]}: {round(preds[0][i]*100, 2)}%")

1/1 [==============================] - 0s 26ms/step
궁: 56.98%
극장: 0.01%
노을: 0.0%
놀이공원: 0.02%
단풍: 0.02%
도심하천: 0.02%
돌담길: 0.0%
번화가: 0.0%
벚꽃: 0.26%
빌딩: 0.37%
아이스링크: 0.0%
아쿠아리움: 0.01%
야경: 0.03%
워터파크: 0.0%
유람선: 0.04%
자연: 0.0%
전통문화: 0.11%
철로: 0.01%
케이블카: 0.0%
타워: 41.84%
트리: 0.28%
호수공원: 0.0%
